<a href="https://colab.research.google.com/github/jsaraujo5081/CursoPythonConfenalco/blob/notebooks/04AnalisisDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISIS DE DATOS EN PYTHON

## Contenido

* Importar data frame
* Ajuste preliminar de variables
* Análisis estadístico y agregación
* Ingeniería de variables

## Importar data frame 

In [2]:
#Cargar librerias para análisis de datos
import pandas as pd
import numpy as np

In [26]:
#Importar data frame
filepath = "https://raw.githubusercontent.com/jsaraujo5081/CursoPythonConfenalco/data/Data_Matriculacion.csv"
df_autos = pd.read_table(filepath, header = 0, sep = ",")

In [ ]:
df_autos.head(5)

In [ ]:
#Estructura del data frame
df_autos.info()

## Ajuste preliminar de variables

In [27]:
#Cambiar tipos de variables (Forma 1)
df_autos["Cod_Vehiculo"] = str(df_autos["Cod_Vehiculo"])

In [28]:
#Forma 2
df_autos["Canton"] = df_autos["Canton"].astype("category")
df_autos["Provincia"] = df_autos["Provincia"].astype("category")
df_autos["Marca"] = df_autos["Marca"].astype("category")
df_autos["Clase"] = df_autos["Clase"].astype("category")
df_autos["Sub_Clase"] = df_autos["Sub_Clase"].astype("category")
df_autos["Combustible"] = df_autos["Combustible"].astype("category")
df_autos["Servicio"] = df_autos["Servicio"].astype("category")

In [ ]:
#Estructura del data frame
df_autos.info()

## Analisis estadístico y agregación

In [ ]:
# Resumen estadístico (Variables numéricas)
df_autos.describe(include = np.number)

In [ ]:
#Correlacion entre Cilindraje y Avaluo
df_cilaval = df_autos[["Cilindraje","Valor_Avaluo"]]
df_cilaval.corr()

In [ ]:
#Resumen estadístico (Variables no numéricas)
df_autos.describe(include = [object,"category"])

In [ ]:
#Serie con conteo por cantón
frec_canton = df_autos["Canton"].value_counts()

#Transformar serie a data frame creando primero un diccionario
d_frec_canton = {"Canton":frec_canton.index,"Frecuencia":frec_canton}
df_frec_canton = pd.DataFrame(d_frec_canton)
df_frec_canton.index = range(0,len(frec_canton))

#Incorporar frecuencia relativa
df_frec_canton["Frec_Relativa"] = np.round(100*df_frec_canton.Frecuencia/np.sum(df_frec_canton.Frecuencia),2)

#Ver resultado
df_frec_canton

In [ ]:
#Conteos o tablas de frecuencia por provincia
frec_provincia = df_autos["Provincia"].value_counts()
d_frec_provincia = {"Provincia":frec_provincia.index,"Frecuencia":frec_provincia}
df_frec_provincia = pd.DataFrame(d_frec_provincia)
df_frec_provincia.index = range(0,len(frec_provincia))
df_frec_provincia["Frec_Relativa"] = np.round(100*df_frec_provincia.Frecuencia/np.sum(df_frec_provincia.Frecuencia),2)
df_frec_provincia

In [84]:
#Otras tablas de frecuencia que desee para variables categóricas

In [ ]:
#Obtener el avalúo promedio por año y mes
df_autos.groupby(["Periodo","Mes"], as_index=False)["Valor_Avaluo"].mean()

In [ ]:
#Obtener para cada periodo y mes el promedio del avaluo y el conteo de vehículos
df_autos.groupby(["Periodo","Mes"], as_index=False).agg({"Valor_Avaluo":"mean","Cod_Vehiculo":"count"})

In [ ]:
#Crear tabla pivote entre paises y clase que cuente casos
df_pais_clase = df_autos.pivot_table("Cod_Vehiculo", index = "Pais_Origen", columns = "Clase", aggfunc = "count")
df_pais_clase

In [ ]:
#Construir tabla pivote por clase y combustible pero con el promedio de avalúo
df_clase_comb = df_autos.pivot_table("Valor_Avaluo", index = "Clase", columns = "Combustible", aggfunc = "mean")
df_clase_comb

## Ingeniería de variables


In [91]:
# Crear variable que se llama "Grupo_Origen" tal que: 
#Si Pais es Ecuador sea "Nacional", si es Colombia, Perú sea "CAN", Caso Contrario sea "Exterior"
lista_aux = []
for i in range(0,len(df_autos)):
  if df_autos["Pais_Origen"][i] == "ECUADOR":
    lista_aux.append("Nacional")
  elif df_autos["Pais_Origen"][i] == "COLOMBIA" or df_autos["Pais_Origen"][i] == "PERU":
    lista_aux.append("CAN")
  else:
    lista_aux.append("Exterior")

df_autos["Grupo_Origen"] = lista_aux

In [ ]:
#Ver un conteo de frecuencias de esta variable creada
frec_origen = df_autos["Grupo_Origen"].value_counts()
d_frec_origen = {"Grupo_Origen":frec_origen.index,"Frecuencia":frec_origen}
df_frec_origen = pd.DataFrame(d_frec_origen)
df_frec_origen.index = range(0,len(frec_origen))
df_frec_origen["Frec_Relativa"] = np.round(100*df_frec_origen.Frecuencia/np.sum(df_frec_origen.Frecuencia),2)
df_frec_origen

In [ ]:
#Crear variable "Rango_Avaluo" que contenga 3 rangos de avaluo tal que:
#Si clase es Automovil, Camioneta o Jeep:
#Si Avaluo es menor a 15000 sea "Bajo", Si esta entre 15000 y 40000 sea "Medio", Caso contrario sea "Alto"
#Para cualquier otra clase el Rango es "No Aplica"

#Construir tabla de frecuencias de esta variable creada

In [ ]:
#Identificar casos atipicos de avaluo en automóviles

#Tabla auxiliar filtrando solamente automóviles en Clase
df_auxiliar = df_autos[df_autos["Clase"] == "AUTOMOVIL"]
df_auxiliar.head()

#Obtener límites para identificar casos atípicos
iqr = df_auxiliar["Valor_Avaluo"].quantile(0.75) - df_auxiliar["Valor_Avaluo"].quantile(0.25)
lim_sup = df_auxiliar["Valor_Avaluo"].quantile(0.75) + 1.5*iqr
lim_inf = df_auxiliar["Valor_Avaluo"].quantile(0.25) - 1.5*iqr
print(lim_sup)
print(lim_inf)

In [116]:
#Crear columna que identifique casos atípicos
lista_aux = []
for i in range(0,len(df_autos)):
  if df_autos["Clase"][i] == "AUTOMOVIL":
    if df_autos["Valor_Avaluo"][i] > lim_sup or  df_autos["Valor_Avaluo"][i] < lim_inf:
      lista_aux.append("ATIPICO")
    else:
      lista_aux.append("OK")
  else:
    lista_aux.append("OK")

df_autos["Caso"] = lista_aux

In [ ]:
#Contar y promedio avaluo por caso
df_autos.groupby(["Caso"], as_index = False).agg({"Cod_Vehiculo":"count","Valor_Avaluo":"mean"})

In [ ]:
#Sustituir casos atipicos por un valor de avaluo mediano
avaluo_ref = df_auxiliar["Valor_Avaluo"].median()

for i in range(0,len(df_autos)):
  if df_autos["Caso"][i] == "ATIPICO":
    df_autos["Valor_Avaluo"][i] = avaluo_ref
  else:
    continue

In [ ]:
#Contar y promedio avaluo por caso
df_autos.groupby(["Caso"], as_index = False).agg({"Cod_Vehiculo":"count","Valor_Avaluo":"mean"})